In [ ]:
!wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz

--2021-11-05 13:18:52--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144903 (2.0M) [application/x-gzip]
Saving to: ‘kddcup.data_10_percent.gz’

kddcup.data_10_perc 100%[===================>]   2.04M  1.94MB/s    in 1.1s    

2021-11-05 13:18:53 (1.94 MB/s) - ‘kddcup.data_10_percent.gz’ saved [2144903/2144903]



In [ ]:
!gzip -d kddcup.data_10_percent.gz

In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

In [ ]:
 col_names = ["duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", 
"land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", 
"num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", 
"num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", 
"is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", 
"rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", 
"srv_diff_host_rate", "dst_host_count","dst_host_srv_count", "dst_host_same_srv_rate", 
"dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", 
"dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", 
"dst_host_srv_rerror_rate", "label"]

In [ ]:
with open('./kddcup.data_10_percent', 'r') as file:
  raw_data = pd.read_csv(file, names=col_names)

In [ ]:
# 데이터 불균형 주의...
raw_data['label'].value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [ ]:
unique_labels = raw_data['label'].unique()
unique_labels

array(['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], dtype=object)

In [ ]:
selected_data = pd.DataFrame()
for label in unique_labels:
  selected_data = pd.concat([selected_data, raw_data.loc[raw_data['label'] == label][:200]])
selected_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.00,0.00,0.11,0.0,0.0,0.00,0.00,0.00,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.00,0.00,0.05,0.0,0.0,0.00,0.00,0.00,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.00,0.00,0.03,0.0,0.0,0.00,0.00,0.00,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.00,0.00,0.03,0.0,0.0,0.00,0.00,0.00,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.00,0.00,0.02,0.0,0.0,0.00,0.00,0.00,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142477,0,tcp,ftp_data,SF,0,5636,0,0,0,0,0,1,2,0,0,2,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,41,1.00,0.00,1.00,0.1,0.0,0.00,0.00,0.00,rootkit.
148154,61,tcp,telnet,SF,294,3929,0,0,0,0,0,1,0,1,0,4,1,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,4,0.02,0.02,0.00,0.0,0.0,0.25,0.73,0.25,rootkit.
397011,0,udp,other,SF,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,1,0.00,0.02,0.00,0.0,0.0,0.00,0.00,0.00,rootkit.
452001,0,udp,other,SF,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.00,0.00,1.00,0.0,0.0,0.00,0.00,0.00,rootkit.


In [ ]:
selected_data['label'].value_counts()


portsweep.          200
normal.             200
smurf.              200
nmap.               200
pod.                200
teardrop.           200
ipsweep.            200
back.               200
satan.              200
warezclient.        200
neptune.            200
guess_passwd.        53
buffer_overflow.     30
land.                21
warezmaster.         20
imap.                12
rootkit.             10
loadmodule.           9
ftp_write.            8
multihop.             7
phf.                  4
perl.                 3
spy.                  2
Name: label, dtype: int64

In [ ]:
print(selected_data['num_outbound_cmds'].value_counts())
print(selected_data['is_host_login'].value_counts())

0    2379
Name: num_outbound_cmds, dtype: int64
0    2379
Name: is_host_login, dtype: int64


In [ ]:
labels = selected_data['label'].to_numpy()
labels


array(['normal.', 'normal.', 'normal.', ..., 'rootkit.', 'rootkit.',
       'rootkit.'], dtype=object)

In [ ]:
selected_data['label'], _ = selected_data['label'].factorize()
selected_data = pd.get_dummies(selected_data)
selected_data

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,protocol_type_icmp,...,service_gopher,service_hostnames,service_http,service_http_443,service_imap4,service_link,service_login,service_mtp,service_name,service_netstat,service_nnsp,service_nntp,service_other,service_pm_dump,service_pop_3,service_private,service_remote_job,service_rje,service_smtp,service_ssh,service_sunrpc,service_supdup,service_systat,service_telnet,service_tim_i,service_time,service_uucp,service_vmnet,service_whois,flag_OTH,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.00,0.00,0.11,0.0,0.0,0.00,0.00,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.00,0.00,0.05,0.0,0.0,0.00,0.00,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.00,0.00,0.03,0.0,0.0,0.00,0.00,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.00,0.00,0.03,0.0,0.0,0.00,0.00,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.00,0.00,0.02,0.0,0.0,0.00,0.00,0.00,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142477,0,0,5636,0,0,0,0,0,1,2,0,0,2,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,41,1.00,0.00,1.00,0.1,0.0,0.00,0.00,0.00,22,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
148154,61,294,3929,0,0,0,0,0,1,0,1,0,4,1,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,4,0.02,0.02,0.00,0.0,0.0,0.25,0.73,0.25,22,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
397011,0,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255,1,0.00,0.02,0.00,0.0,0.0,0.00,0.00,0.00,22,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
452001,0,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,1.00,0.00,1.00,0.0,0.0,0.00,0.00,0.00,22,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
split_data = selected_data.copy()

split_data.drop(['num_outbound_cmds'], axis=1, inplace=True)
split_data.drop(['is_host_login'], axis=1, inplace=True)

corr_matrix = selected_data.corr()
corr_label = corr_matrix["label"].sort_values(ascending=False)

for index, num in corr_label.items():
  if abs(num) < 0.1:
    split_data.drop([index], axis=1, inplace=True)

split_data

,duration,dst_bytes,wrong_fragment,hot,is_guest_login,srv_count,rerror_rate,srv_rerror_rate,diff_srv_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,protocol_type_icmp,protocol_type_tcp,service_eco_i,service_ecr_i,service_ftp,service_ftp_data,service_http,service_other,service_telnet,flag_REJ,flag_S0,flag_SH
0,0,5450,0,0,0,8,0.0,0.0,0.0,9,1.00,0.00,0.00,0.00,0,0,1,0,0,0,0,1,0,0,0,0,0
1,0,486,0,0,0,8,0.0,0.0,0.0,19,1.00,0.00,0.00,0.00,0,0,1,0,0,0,0,1,0,0,0,0,0
2,0,1337,0,0,0,8,0.0,0.0,0.0,29,1.00,0.00,0.00,0.00,0,0,1,0,0,0,0,1,0,0,0,0,0
3,0,1337,0,0,0,6,0.0,0.0,0.0,39,1.00,0.00,0.00,0.00,0,0,1,0,0,0,0,1,0,0,0,0,0
4,0,2032,0,0,0,6,0.0,0.0,0.0,49,1.00,0.00,0.00,0.00,0,0,1,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142477,0,5636,0,0,0,1,0.0,0.0,0.0,41,1.00,0.00,0.00,0.00,22,0,1,0,0,0,1,0,0,0,0,0,0
148154,61,3929,0,0,0,1,0.0,0.0,0.0,4,0.02,0.02,0.73,0.25,22,0,1,0,0,0,0,0,0,1,0,0,0
397011,0,0,0,0,0,1,0.0,0.0,0.0,1,0.00,0.02,0.00,0.00,22,0,0,0,0,0,0,0,1,0,0,0,0
452001,0,4,0,0,0,1,0.0,0.0,0.0,1,1.00,0.00,0.00,0.00,22,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
data = split_data.drop('label', axis=1)
data

,duration,dst_bytes,wrong_fragment,hot,is_guest_login,srv_count,rerror_rate,srv_rerror_rate,diff_srv_rate,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type_icmp,protocol_type_tcp,service_eco_i,service_ecr_i,service_ftp,service_ftp_data,service_http,service_other,service_telnet,flag_REJ,flag_S0,flag_SH
0,0,5450,0,0,0,8,0.0,0.0,0.0,9,1.00,0.00,0.00,0.00,0,1,0,0,0,0,1,0,0,0,0,0
1,0,486,0,0,0,8,0.0,0.0,0.0,19,1.00,0.00,0.00,0.00,0,1,0,0,0,0,1,0,0,0,0,0
2,0,1337,0,0,0,8,0.0,0.0,0.0,29,1.00,0.00,0.00,0.00,0,1,0,0,0,0,1,0,0,0,0,0
3,0,1337,0,0,0,6,0.0,0.0,0.0,39,1.00,0.00,0.00,0.00,0,1,0,0,0,0,1,0,0,0,0,0
4,0,2032,0,0,0,6,0.0,0.0,0.0,49,1.00,0.00,0.00,0.00,0,1,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142477,0,5636,0,0,0,1,0.0,0.0,0.0,41,1.00,0.00,0.00,0.00,0,1,0,0,0,1,0,0,0,0,0,0
148154,61,3929,0,0,0,1,0.0,0.0,0.0,4,0.02,0.02,0.73,0.25,0,1,0,0,0,0,0,0,1,0,0,0
397011,0,0,0,0,0,1,0.0,0.0,0.0,1,0.00,0.02,0.00,0.00,0,0,0,0,0,0,0,1,0,0,0,0
452001,0,4,0,0,0,1,0.0,0.0,0.0,1,1.00,0.00,0.00,0.00,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
#변경 시도
scaler = MinMaxScaler()
#scaler = StandardScaler()
data = scaler.fit_transform(data)
data[0]

array([0.        , 0.00105713, 0.        , 0.        , 0.        ,
       0.01372549, 0.        , 0.        , 0.        , 0.03149606,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [ ]:
max = 0
max_index = 0
max_cluster = 0
for i in range(1, 10):
  tem_data = PCA(n_components=i).fit_transform(data)
  n_clusters = len(set(unique_labels))
  for j in range(2, n_clusters+1):
    kmeans = KMeans(n_clusters=j)
    kmeans.fit(tem_data)
    label, _ = pd.factorize(labels)
    kmeans_label = kmeans.labels_
    if metrics.v_measure_score(label, kmeans_label) > max:
      max = metrics.v_measure_score(label, kmeans_label)
      max_index = i
      max_cluster = j
print(max_index)
print(max_cluster)
print(max)

9
17
0.8542636106285751


In [ ]:
reduced_data = PCA(n_components=9).fit_transform(data)
kmeans = KMeans(n_clusters=14)
kmeans.fit(reduced_data)
label, _ = pd.factorize(labels)
kmeans_label = kmeans.labels_
print("Homogeneity: %0.3f" % metrics.homogeneity_score(label, kmeans_label))
print("Completeness: %0.3f" % metrics.completeness_score(label, kmeans_label))
print("V-measure: %0.3f" % metrics.v_measure_score(label, kmeans_label))
print()

Homogeneity: 0.774
Completeness: 0.870
V-measure: 0.819



In [ ]:
reduced_data = PCA(n_components=9).fit_transform(data)
kmeans = KMeans(n_clusters=17)
kmeans.fit(reduced_data)
label, _ = pd.factorize(labels)
kmeans_label = kmeans.labels_
print("Homogeneity: %0.3f" % metrics.homogeneity_score(label, kmeans_label))
print("Completeness: %0.3f" % metrics.completeness_score(label, kmeans_label))
print("V-measure: %0.3f" % metrics.v_measure_score(label, kmeans_label))
print()

Homogeneity: 0.840
Completeness: 0.870
V-measure: 0.855



최적
standard scaler
PCA 9, n_clusters 17

## DBSCAN
minpts 최소 구성 클러스터  
eps 바운더리  
core 원 안에 다른 점들이 minpts이상인 클러스터  
noise 원 안에 minpts만큼 없고 보더 처럼 코어에 걸려있지도 않음  
border 원 안에 minpts만큼 없고 코어에 걸려있음  

In [ ]:
max = 0
max_index = 0
max_samples = 0
for i in range(1, 15):
  tem_data = PCA(n_components=i).fit_transform(data)

  for j in range(2, 20):
    dbscan = DBSCAN(eps = 0.5, min_samples=j)
    dbscan.fit(tem_data)
    dbscan_label = dbscan.labels_
    dbscan_unique_labels = list(set(dbscan_label))
    n_clusters_ = len(set(dbscan_label)) - (1 if -1 in dbscan_label else 0)
    
    if max < metrics.v_measure_score(labels, dbscan_label):
      max_index = i
      max = metrics.v_measure_score(labels, dbscan_label)
      max_samples = j

print(max_index)
print(max_samples)
print(max)

12
2
0.8358933972172136


In [ ]:
tem_data = PCA(n_components=14).fit_transform(data)
dbscan = DBSCAN(eps = 0.5, min_samples=2)
dbscan.fit(tem_data)
dbscan_label = dbscan.labels_
dbscan_unique_labels = list(set(dbscan_label))
n_clusters_ = len(set(dbscan_label)) - (1 if -1 in dbscan_label else 0)
print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, dbscan_label))
print("Completeness: %0.3f" % metrics.completeness_score(labels, dbscan_label))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, dbscan_label))

Estimated number of clusters: 34
Homogeneity: 0.869
Completeness: 0.823
V-measure: 0.845


In [ ]:
reduced_data = PCA(n_components=14).fit_transform(data)
dbscan = DBSCAN(eps = 0.5, min_samples=2)
dbscan.fit(reduced_data)
dbscan_label = dbscan.labels_
dbscan_unique_labels = list(set(dbscan_label))
n_clusters_ = len(set(dbscan_label)) - (1 if -1 in dbscan_label else 0)
print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, dbscan_label))
print("Completeness: %0.3f" % metrics.completeness_score(labels, dbscan_label))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, dbscan_label))

Estimated number of clusters: 34
Homogeneity: 0.869
Completeness: 0.823
V-measure: 0.845


 최적  
0.1 삭제  
min-max scaler  
PCA 14, eps 0.5, min_samples 2  
0.845